In [1]:
import numpy as np

In [9]:
M = 300 #number of random walkers
R = np.zeros((2*M,3))
for m in range(M):
    for d in range(3):
        R[m,d] = np.random.uniform(-0.5, 0.5)

(300, 3)

In [ ]:
def greens_function(config, old_config, alpha, beta, kappa, gamma, delta_t):
    norm_diff = np.linalg.norm(config - old_config)
    return (1/np.sqrt(4*np.pi* gamma*delta_t)) * np.exp(-norm_diff**2 / (4*gamma*delta_t))

In [ ]:
def psi_T(vec_r_1, vec_r_2, alpha, kappa = 2, beta = 0.5):
    r_1 = np.linalg.norm(vec_r_1)
    r_2 = np.linalg.norm(vec_r_2)
    vec_r12 = vec_r_2 - vec_r_1
    r_12 = np.linalg.norm(vec_r12)
    pref = beta*r_12/(1+alpha*r_12)
    psi = np.exp(-kappa*r_1)*np.exp(kappa*r_2)*np.exp(pref)
    return psi

def energy_local(vec_r_1, vec_r_2, alpha, kappa = 2, beta = 0.5):
    r_1 = np.sqrt(np.sum([x**2 for x in vec_r_1]))
    r_2 = np.sqrt(np.sum([x**2 for x in vec_r_2]))
    vec_r12 = vec_r_2 - vec_r_1
    r_12 = np.sqrt(np.sum([x**2 for x in vec_r12]))
    u = 1+alpha*r_12
    scalar_product = np.dot(vec_r_1/r_1 - vec_r_2/r_2, vec_r12/r_12)
    E_L = (kappa-2)/r_1 + (kappa-2)/r_2 + (1-2*beta/u**2) +2*beta*alpha/u**3 - kappa**2 - (beta**2)/u**4 +(kappa*beta/u**2)*scalar_product
    return E_L

def Fokker_planck(vec_r_1, vec_r_2, alpha, kappa = 2, beta = 0.5):
    r_1 = np.linalg.norm(vec_r_1)
    r_2 = np.linalg.norm(vec_r_2)
    vec_r12 = vec_r_2 - vec_r_1
    r_12 = np.linalg.norm(vec_r12)
    a = 1 + alpha*r_12
    factor = beta*(((a/r_12)-alpha)/a**2)
    force_1 = 2*(-kappa*(vec_r_1/r_1)+factor*(vec_r_1-vec_r_2))
    force_2 = 2*(-kappa*(vec_r_2/r_2)+factor*(vec_r_2-vec_r_1))
    return np.array([force_1, force_2])

def metropolis_algorithm(initial_state, num_trials, beta=1, list_seeds=None):
    initial_r_1 = initial_state[:M]
    initial_r_2 = initial_state[M:]
    sequence_states = [initial_state]
    #Todo: Implement random selection of R for metropolis step
    for n in range(num_trials):
        next_sequence = metropolis_step(sequence_states[-1], beta)
        sequence_states.append(next_sequence)
    return np.array(sequence_states), np.arange(num_trials+1) #sequences and corresponding time steps
    
def metropolis_step(current_state, alpha, beta, s, kappa, delta_t, seed = None):
    #np.random.seed(seed)
    trial_configuration = current_state.copy()
    current_r_1 = current_state[:M]
    current_r_2 = current_state[M:]
    trial_r_1 = trial_configuration[:M]
    trial_r_2 = trial_configuration[M:]
    which_particle = np.random.choice([0,M])
    for i in range(M):
        trial_configuration[i+which_particle] += Fokker_planck()*delta_t/2
        trial_configuration[i+which_particle] += np.random.uniform(low=-s/2, high=s/2, size=3)*np.sqrt(delta_t)
    
    trial_acceptance_prob = np.min([1, (psi_T(trial_r_1, trial_r_2, alpha, kappa, beta)/psi_T(current_r_1, current_r_2, alpha, kappa, beta))**2])
    r_accept = np.random.rand()
    if r_accept < trial_acceptance_prob:
        new_state = trial_configuration
    else:
        new_state = current_state
    return np.array(new_state)
